## REVISAT
REVISAT is a script that allows reviewing a dataset, in draft, before being published, to ensure compliance with good open access practices. It is a first version, and as the repository software is updated and/or metadata is updated, the script will be changed accordingly.
If you as a user have any doubts about the operation, proposal, or suggestion for improvement and want to incorporate it into the script, please write to us at: rdr-contacte@csuc.cat

Last updated: 2023-11-14

In [ ]:
# @title Install or update libraries (Click execution button &#x25B6; )
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Function to install required packages
def install_packages(b):
    clear_output(wait=True)
    !pip install --upgrade pip -q
    !pip --upgrade tensorflow-probability -q
    !pip install pyDataverse -q
    !pip install selenium -q
    print("S'han descarregat o actualitzat les llibreries.")

# Displaying installation message
display(HTML("<p style='font-size:14px;'><b>Feu clic al botó següent per instal·lar les llibreries.</b></p>"))

# Creating installation button
install_button = widgets.Button(description='Instal·lar llibreries')
install_button.on_click(install_packages)

# Displaying the installation button
display(install_button)

In [ ]:
# @title Introduir DOI (doi:10.34810/dataXXX), el token i el nom complet de la institució. Clicar botó d'executar cel·la &#x25B6;
from datetime import date
from pyDataverse.api import NativeApi, DataAccessApi
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import sys
import numpy as np
import os
from collections import Counter
import textwrap
import pprint
from IPython.display import HTML, display
# Provide input values
doi = ""  # @param {type:"string"}
token = ""  # @param {type:"string"}
centre= "" # @param {type:"string"}
driver = None ## triar (webdriver.Chrome(), webdriver.Firefox() or None) per evaluar el daset a F-uji. Trieu None si useu l'script a Colab.
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)
def Meta(doi, token, driver, centro):
    today = date.today()
    print("Data:", today)
#    print("Dataset DOI: ", doi)

    base_url = 'https://dataverse.csuc.cat/'
    api = NativeApi(base_url,token)
    Metadata = api.get_dataset(doi)
    c_doi = Metadata.json()["data"]["latestVersion"]['datasetPersistentId']

    fields_metadata = Metadata.json()["data"]["latestVersion"]["metadataBlocks"]["citation"]["fields"]
    len_metadata = len(fields_metadata)
    Metadata_min_req = ['title', 'datasetContact', 'dsDescription', 'keyword', 'subject', 'kindOfData', 'author']

    metadata_repositori = []
    #today = date.today()

    i = 0
    while i <= len_metadata:
        metadata_repositori.append(fields_metadata[i]["typeName"])
        i +=1
        if i == len_metadata:
            break


    intersect_metadata = list(set(metadata_repositori) & set(Metadata_min_req))# Comprobamos la coincidencia  de campos
    same_metadata = len(list(set(Metadata_min_req)^set(intersect_metadata)))# 0 --> have minimum of metadata

    print('\nConté les metadades mínimes obligatòries?')
    Metadata_min = []  #####################################################################################
    Metadata_faltante = []
    if same_metadata !=0:
        #print("NO\nFalta la medata:", list(set(Metadata_min_req)^set(intersect_metadata)))
        Metadata_faltante.append(list(set(Metadata_min_req)^set(intersect_metadata)))
        print("NO", Metadata_faltante)
    else:
        #print("SÍ")
        #Metadata_min.append("SÍ")
        print("SÍ")

    index_title = metadata_repositori.index('title')
    print("\nTítol dataset:\n{}\n".format( fields_metadata[index_title]["value"]))
    titol = fields_metadata[index_title]["value"]
    titol_1 = titol.split(":")
    #print(titol[22:])
    #print(titol_1)

    ###### Related publication

    relatedpublication = 'publication'
    #print("RELATION PUBLICATION:")
    print("En el cas que el dataset tingui una publicació relacionada, inclou la citació?")

    if relatedpublication in metadata_repositori:
        print("SÍ")
        Rel_pub = []

        index_publication = metadata_repositori.index(relatedpublication)
        for i in fields_metadata[index_publication]["value"]:
            Rel_pub.append(i["publicationCitation"]["value"])
            print(i["publicationCitation"]["value"])

        #print("Related publication:", Rel_pub)

        if ("Replication Data for" in titol_1) & (len(titol_1) > 1):
            only_title = titol[21:]
            print("\nEl títol inclou: Replication data for")
            ##Comprobamos que tengan el mismo titulo:
            for i in Rel_pub[0].split("."):
                #print (i)
                if (only_title.casefold() == i.casefold()):
                    print("Els títols coincideixen")
        else: print("\nNo és rèplica de l'article")

    else: print("\nNo té publicacions relacionades")


    index_author = metadata_repositori.index('author')
    author_id = []
    afiliacion = []
    institucion = []

    for i in range(len(fields_metadata[index_author]["value"])):
        #print(list(fields_metadata[index_author]["value"][i].keys()))
        for j in (list(fields_metadata[index_author]["value"][i].keys())):
            #print(j)
            if (j == 'authorAffiliation'):
                afiliacion.append(fields_metadata[index_author]["value"][i]['authorAffiliation']["value"])

            if (j == 'authorIdentifier'):
                author_id.append("SÍ")


    #university = 'Universitat Autònoma de Barcelona'
    #university = 'Universitat Politècnica de Catalunya'
    ##At least one of the authors belongs to the institution where it is deposited
    for i in afiliacion:
        ins = i.split(".")
        if centro in ins:
            institucion.append("SÍ")
        else: institucion.append("NO")


    print("\nAlmenys un/a dels/les autors/es pertany a la institució on es diposita: ", "SÍ" if "SÍ" in institucion else "NO")
    ## At least one of the authors reports their ORCID
    print("Almenys un/a dels/les autors/es informa del seu ORCID?")
    print("ORCID: ", "SÍ" if "SÍ" in author_id else "NO")


    ###### Description
    index_descripcion = metadata_repositori.index('dsDescription')
    descripcion = []
    print("\nDescripció")
    descripcion.append(fields_metadata[index_descripcion]["value"][0]['dsDescriptionValue']["value"])
    print(descripcion[0])



    print("\nFormat de fitxers")
    total_files = len(Metadata.json()['data']['latestVersion']['files'])
    files = []
    files_extension = []

    for i in range(total_files):
        #if i == "Readme.txt" or i == "readme.txt":
            #print(os.path.splitext(Metadata.json()['data']['latestVersion']['files'][i]['dataFile']['filename'])[1])
            #print(Metadata.json()['data']['latestVersion']['files'][i]['dataFile']['filename'])
            files_extension.append(os.path.splitext(Metadata.json()['data']['latestVersion']['files'][i]['dataFile']['filename'])[1])
            files.append(Metadata.json()['data']['latestVersion']['files'][i]['dataFile']['filename'])


    print(Counter(files_extension))

    lowercase_files = list(map(lambda x: x.lower(), files))
    for j in lowercase_files:
        if j == "readme.txt":
            print("Sí que conté el fitxer readme.txt")


    ###### License
    print("\nLlicència:")
    if 'license' in Metadata.json()["data"]['latestVersion'].keys():
        print(Metadata.json()["data"]['latestVersion']["license"]["name"])
    else:
        print(Metadata.json()["data"]['latestVersion']['termsOfUse'])


###### F-UJI
    if driver is None:
        print("\nAvalueu el dataset manualment a F-UJI: https://www.f-uji.net/")

    else:
        driver.get("https://www.f-uji.net/")
        driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/p/a').click()
        driver.find_element(By.XPATH, '//*[@id="pid"]').send_keys(doi)
        driver.find_element(By.XPATH, '//*[@id="assessment_form"]/div/form/div[4]/button').click()
Meta(doi, token, driver,centre)